<a href="https://colab.research.google.com/github/nsstnaka/machine_learning_handson/blob/master/knowledge_graph_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習ハンズオン（ナレッジグラフ埋め込み）

## 1. ハンズオンの概要
TransEでナレッジグラフの埋め込み表現を学習し、未知の質問に対して推論を試します。

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import math

## 2. データ取得

### 2.1. データのダウンロード・展開
データは"Cross-lingual entity alignment via joint attribute-preserving embedding"の論文の実験で使用したものを使います。

In [ ]:
!wget -nc https://github.com/nju-websoft/JAPE/raw/master/data/dbp15k.tar.gz
!tar zxf dbp15k.tar.gz

### 2.2. ファイル読込

In [ ]:
df = pd.read_csv('dbp15k/ja_en/s_triples', sep='\t', names=['head', 'relation', 'tail'])
print(len(df))
df.head(10)

## 3. 前処理

### 3.1. 各エンティティ・リレーションの`http://...`を除去

In [ ]:
df = df.applymap(lambda x: x.split('/')[-1])
df.head(10)

### 3.2. 名称→通し番号への変換
エンティティおよびリレーションの名称を番号に変換します。

In [ ]:
entity_set = set(df['head']) | set(df['tail'])
relation_set = set(df['relation'])
len(entity_set), len(relation_set)

In [ ]:
entity_idx_dic = {name: idx for idx, name in enumerate(entity_set)}
entity_name_dic = {idx: name for name, idx in entity_idx_dic.items()}
relation_idx_dic = {name: idx for idx, name in enumerate(relation_set)}
relation_name_dic = {idx: name for name, idx in relation_idx_dic.items()}

In [ ]:
df['head_idx'] = df['head'].apply(lambda x: entity_idx_dic[x])
df['relation_idx'] = df['relation'].apply(lambda x: relation_idx_dic[x])
df['tail_idx'] = df['tail'].apply(lambda x: entity_idx_dic[x])
df.head(10)

### 3.3. テスト用のトリプルの選定
推論のテストに使用するトリプルを1つ選び、訓練データから除去します。

In [ ]:
df[(df['head'] == 'イギリス') & (df['relation'] == '首都')]

In [ ]:
test_head = entity_idx_dic['イギリス']
test_relation = relation_idx_dic['首都']
test_tail = entity_idx_dic['ロンドン']
test_head, test_relation, test_tail

In [ ]:
df.drop(4917, axis=0, inplace=True)

## 4. 学習

### 4.1. 各種設定

In [ ]:
EMBEDDING_DIM = 75  # 埋め込みの次元数
TRAIN_BATCH_SIZE = 256  # 学習バッチサイズ
SCORE_FUNC = 'L1'  # 距離の計算方法 'L1'（マンハッタン距離） or 'L2'（ユークリッド距離）
NORMALIZE = True  # ベクトルのL2正規化有無
EPOCHS = 50  # 学習エポック数

### 4.2. 余計な列の除去
訓練に使わない列は除去しておきます。

In [ ]:
df.drop(['head', 'relation', 'tail'], axis=1, inplace=True)
df.head(5)

### 4.3. 訓練データの生成

負例を作る際にヘッドを入れ替えるかテイルを入れ替えるかの確率を生成します。

In [ ]:
head_count = df.groupby('relation_idx')['head_idx'].apply(set).apply(len)
tail_count = df.groupby('relation_idx')['tail_idx'].apply(set).apply(len)
tail_prob = tail_count / (head_count + tail_count)

訓練データのgeneratorの定義

In [ ]:
def data_generator(num_entities, triples, prob):
    triple_set = {(row[0], row[1], row[2]) for row in triples}
    for head, relation, tail in np.random.permutation(triples):
        neg_head = None
        neg_tail = None
        if random.random() > prob[relation]:
            # replace head
            neg_tail = tail
            while True:
                neg_head = random.randrange(num_entities)
                if (neg_head, relation, tail) not in triple_set:
                    break
        else:
            # replace tail
            neg_head = head
            while True:
                neg_tail = random.randrange(num_entities)
                if (head, relation, neg_tail) not in triple_set:
                    break
        yield [head, relation, tail, neg_head, neg_tail]

generatorから`tf.data`のDatasetを作成します。

In [ ]:
train_ds = tf.data.Dataset.from_generator(data_generator, args=[len(entity_idx_dic), df.values, tail_prob], output_types=(tf.int64), output_shapes=(5,)).shuffle(10000).batch(TRAIN_BATCH_SIZE)

### 4.4. 学習モデル構築

距離の算出関数

In [ ]:
def score_func(heads, tails, relations):
    #return tf.square(tf.norm(heads + relations - tails, ord=2, axis=-1))
    if SCORE_FUNC == 'L1':
        return tf.norm(heads + relations - tails, ord=1, axis=-1)
    elif SCORE_FUNC == 'L2':
        return tf.square(tf.norm(heads + relations - tails, ord=2, axis=-1))
    else:
        raise Exception('Invalid SCORE_FUNC:', SCORE_FUNC)

埋め込みの定義と初期化

In [ ]:
bound = 6 / math.sqrt(EMBEDDING_DIM)
initializer = tf.keras.initializers.RandomUniform(minval=-bound, maxval=bound)
entity_embeddings = tf.keras.layers.Embedding(len(entity_idx_dic), EMBEDDING_DIM,
                                              embeddings_initializer=initializer)
relation_embeddings = tf.keras.layers.Embedding(len(entity_idx_dic), EMBEDDING_DIM,
                                                embeddings_initializer=initializer)

学習部分の定義

In [ ]:
margin = 1.0
optimizer = tf.keras.optimizers.Adam()
#optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
        heads = entity_embeddings(inputs[:, 0])
        relations = relation_embeddings(inputs[:, 1])
        tails = entity_embeddings(inputs[:, 2])
        neg_heads = entity_embeddings(inputs[:, 3])
        neg_tails = entity_embeddings(inputs[:, 4])
        if NORMALIZE:
            heads = tf.nn.l2_normalize(heads, axis=-1)
            tails = tf.nn.l2_normalize(tails, axis=-1)
            relations = tf.nn.l2_normalize(relations, axis=-1)
            neg_heads = tf.nn.l2_normalize(neg_heads, axis=-1)
            neg_tails = tf.nn.l2_normalize(neg_tails, axis=-1)
        pos_scores = score_func(heads, tails, relations)
        neg_scores = score_func(neg_heads, neg_tails, relations)
        loss = tf.reduce_sum(tf.maximum(pos_scores + margin - neg_scores, 0.0))
    #print(variables)
    variables = entity_embeddings.trainable_variables + relation_embeddings.trainable_variables
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))
    return loss

学習実行

In [ ]:
for e in range(1, EPOCHS+1):
    total_loss = 0.0
    for batch_data in train_ds:
        loss = train_step(batch_data)
        total_loss += loss.numpy()
    print("Epoch {}: loss={:.6f}".format(e, total_loss))

## 5. 推論実行

学習済みの埋め込みから、テスト用のヘッドとリレーションに該当する埋め込みを取得します。
（後で計算が楽になるように、事前にこれらを足しておきます）

In [ ]:
test_entity_emb = entity_embeddings(test_head)
test_relation_emb = entity_embeddings(test_relation)
if NORMALIZE:
  test_entity_emb = tf.nn.l2_normalize(test_entity_emb)
  test_relation_emb = tf.nn.l2_normalize(test_relation_emb)
target_emb = tf.expand_dims(tf.nn.l2_normalize(entity_embeddings(test_head)) + relation_embeddings(test_relation), axis=0)
target_emb

推論結果の候補となるエンティティ（つまりすべてのエンティティ）の埋め込み表現を取得します。

In [ ]:
all_entity_embs = entity_embeddings(np.array(range(len(entity_idx_dic))))
if NORMALIZE:
  all_entity_embs = tf.nn.l2_normalize(all_entity_embs, axis=-1)
all_entity_embs

$head + relation - tail$の距離を算出（この距離が最も近いものが最有力候補となる）し、距離が近い順に候補のエンティティを並べ替えます。

In [ ]:
if SCORE_FUNC == 'L1':
  nearest_entities = tf.argsort(tf.linalg.norm(target_emb - all_entity_embs, ord=1, axis=-1)).numpy()
else:  # L2
  nearest_entities = tf.argsort(tf.linalg.norm(target_emb - all_entity_embs, ord=2, axis=-1)).numpy()

正解のテイルが何番目に入っているかを確認します（この順位が小さいほうが精度が良いことになります）

In [ ]:
rank = 0
for i in range(len(entity_idx_dic)):
  if nearest_entities[i] == test_tail:
    rank = i+1
    break
print(rank)